In [1]:
#import designs_setup as ds
import numpy as np

In [2]:
def lat_toCluster(I,J,k,q1=0,q2=0,divides = False):
  '''
  Returns the cluster assignment (s,t) of unit(s) (i,j) for i in I and j in J

  i (int or np.array): row position of unit on n by n lattice (or array of row positions)
  j (int or np.array): column position of unit on n by n lattice (or array of col positions)
  k (int): typical cluster side length (each cluster is itself a k by k grid graph with k << n)
  q1 (int): "origin" row position marking the end (inclusive) of first cluster
  q2 (int): "origin" col position marking the end (inclusive) of first cluster
  divides (boolean): if k divides n should be set to True
  '''
  if divides:
    s = np.floor(I/k)
    t = np.floor(J/k)
  else:
    s = np.ceil((I-q1)/k)
    t = np.ceil((J-q2)/k)
  
  return s.astype(int),t.astype(int)

In [5]:
def cluster_neighborhood(A,i,k):
    '''
    Given an adjacency matrix A and unit i, 
    returns a list of labels corresponding to the clusters adjacent to i
    k = "typical" cluster side length
    '''
    pop_size = np.shape(A)[0] 
    n = int(np.sqrt(pop_size))  # population size is n^2
    nc = int(np.ceil(n/k)**2)   # number of clusters is nc^2

    # get indicies of i's neighbors (nonzero entries in i-th row of A)
    neighbors = np.flatnonzero(A[i,:])

    # We have nc^2 clusters represented by an nc x nc grid
    # We have labels (s,t) in [nc] x [nc] for each cluster
    # We also have labels k in [nc^2] for each cluster
    # Given (s,t), k = nc*s + t. Given k, (s,t)=(np.floor(k/nc),k%nc).
    # For each neighbor, get their cluster assignment (s,t)
    cluster_assignments = []
    for x in np.nditer(neighbors):
        print("neighbor:", x)
        # get the (i,j) coordinate of this neighbor on the population lattice [n] x [n]
        i = int(np.floor(x/n))
        j = x % n
        print("i,j= ", (i,j))
        s,t = lat_toCluster(i,j,k,divides=(k%n==0))
        print("s,t= ", (s,t))
        cluster_assignments.append((s,t))
    
    # remove duplicates
    cluster_assignments = list(set(cluster_assignments))

    return cluster_assignments

In [4]:
A = np.array([[1,1,1,0],[1,1,0,1],[1,0,1,1],[0,1,1,1]])
print(A)

[[1 1 1 0]
 [1 1 0 1]
 [1 0 1 1]
 [0 1 1 1]]


In [5]:
l = cluster_neighborhood(A,0,2)
print(l)

[(0, 0)]


In [6]:
A = np.array([[1,1,0,1,0,0,0,0,0],
            [1,1,1,0,1,0,0,0,0],
            [0,1,1,0,0,1,0,0,0],
            [1,0,0,1,1,0,1,0,0],
            [0,1,0,1,1,1,0,1,0],
            [0,0,1,0,1,1,0,0,1],
            [0,0,0,1,0,0,1,1,0],
            [0,0,0,0,1,0,1,1,1],
            [0,0,0,0,0,1,0,1,1]])
print(A)

[[1 1 0 1 0 0 0 0 0]
 [1 1 1 0 1 0 0 0 0]
 [0 1 1 0 0 1 0 0 0]
 [1 0 0 1 1 0 1 0 0]
 [0 1 0 1 1 1 0 1 0]
 [0 0 1 0 1 1 0 0 1]
 [0 0 0 1 0 0 1 1 0]
 [0 0 0 0 1 0 1 1 1]
 [0 0 0 0 0 1 0 1 1]]


In [8]:
k = 2
l0 = cluster_neighborhood(A,0,k)
l1 = cluster_neighborhood(A,1,k)
l8 = cluster_neighborhood(A,8,k)
print(l0)
print(l1)
print(l8)

neighbor: 0
i,j=  (0.0, 0)
s,t=  (0, 0)
neighbor: 1
i,j=  (0.0, 1)
s,t=  (0, 1)
neighbor: 3
i,j=  (1.0, 0)
s,t=  (1, 0)
neighbor: 0
i,j=  (0.0, 0)
s,t=  (0, 0)
neighbor: 1
i,j=  (0.0, 1)
s,t=  (0, 1)
neighbor: 2
i,j=  (0.0, 2)
s,t=  (0, 1)
neighbor: 4
i,j=  (1.0, 1)
s,t=  (1, 1)
neighbor: 5
i,j=  (1.0, 2)
s,t=  (1, 1)
neighbor: 7
i,j=  (2.0, 1)
s,t=  (1, 1)
neighbor: 8
i,j=  (2.0, 2)
s,t=  (1, 1)
[(0, 1), (1, 0), (0, 0)]
[(0, 1), (1, 1), (0, 0)]
[(1, 1)]


### When $k$ divides $n$
First, I'm testing whether my code works when $k | n$. I'm making the choice that when $k | n$ we don't sample a random origin. 

In [11]:
n = 12
k = 3

print('Cluster',':','I ',' J')
for i in range(n//k):
    for j in range(n//k):
        I,J = ds.sqlat_toUnit(i,j,k,n)
        print((i,j),':', I,J)
    print()

Cluster : I   J
(0, 0) : [0 1 2] [0 1 2]
(0, 1) : [0 1 2] [3 4 5]
(0, 2) : [0 1 2] [6 7 8]
(0, 3) : [0 1 2] [ 9 10 11]

(1, 0) : [3 4 5] [0 1 2]
(1, 1) : [3 4 5] [3 4 5]
(1, 2) : [3 4 5] [6 7 8]
(1, 3) : [3 4 5] [ 9 10 11]

(2, 0) : [6 7 8] [0 1 2]
(2, 1) : [6 7 8] [3 4 5]
(2, 2) : [6 7 8] [6 7 8]
(2, 3) : [6 7 8] [ 9 10 11]

(3, 0) : [ 9 10 11] [0 1 2]
(3, 1) : [ 9 10 11] [3 4 5]
(3, 2) : [ 9 10 11] [6 7 8]
(3, 3) : [ 9 10 11] [ 9 10 11]



In [13]:
I = np.array([1,2])
J = np.array([1,2])
s,t = ds.sqlat_toCluster(I,J,k,divides=True)
print(s,t)

[0 0] [0 0]


In [3]:
print('Unit', ':', 'Cluster')
for i in [5,10]:
    for j in range(n):
        s,t = ds.sqlat_toCluster(i,j,k,divides=True)
        print((i,j),':',(s,t))
    print()

Unit : Cluster
(5, 0) : (1, 0)
(5, 1) : (1, 0)
(5, 2) : (1, 0)
(5, 3) : (1, 1)
(5, 4) : (1, 1)
(5, 5) : (1, 1)
(5, 6) : (1, 2)
(5, 7) : (1, 2)
(5, 8) : (1, 2)
(5, 9) : (1, 3)
(5, 10) : (1, 3)
(5, 11) : (1, 3)

(10, 0) : (3, 0)
(10, 1) : (3, 0)
(10, 2) : (3, 0)
(10, 3) : (3, 1)
(10, 4) : (3, 1)
(10, 5) : (3, 1)
(10, 6) : (3, 2)
(10, 7) : (3, 2)
(10, 8) : (3, 2)
(10, 9) : (3, 3)
(10, 10) : (3, 3)
(10, 11) : (3, 3)



In [4]:
k = 4

print('Cluster',':','I ',' J')
for i in range(n//k):
    for j in range(n//k):
        I,J = ds.sqlat_toUnit(i,j,k,n)
        print((i,j),':', I,J)
    print()

Cluster : I   J
(0, 0) : [0 1 2 3] [0 1 2 3]
(0, 1) : [0 1 2 3] [4 5 6 7]
(0, 2) : [0 1 2 3] [ 8  9 10 11]

(1, 0) : [4 5 6 7] [0 1 2 3]
(1, 1) : [4 5 6 7] [4 5 6 7]
(1, 2) : [4 5 6 7] [ 8  9 10 11]

(2, 0) : [ 8  9 10 11] [0 1 2 3]
(2, 1) : [ 8  9 10 11] [4 5 6 7]
(2, 2) : [ 8  9 10 11] [ 8  9 10 11]



In [5]:
print('Unit', ':', 'Cluster')
for i in [3,9]:
    for j in range(n):
        s,t = ds.sqlat_toCluster(i,j,k,divides=True)
        print((i,j),':',(s,t))
    print()

Unit : Cluster
(3, 0) : (0, 0)
(3, 1) : (0, 0)
(3, 2) : (0, 0)
(3, 3) : (0, 0)
(3, 4) : (0, 1)
(3, 5) : (0, 1)
(3, 6) : (0, 1)
(3, 7) : (0, 1)
(3, 8) : (0, 2)
(3, 9) : (0, 2)
(3, 10) : (0, 2)
(3, 11) : (0, 2)

(9, 0) : (2, 0)
(9, 1) : (2, 0)
(9, 2) : (2, 0)
(9, 3) : (2, 0)
(9, 4) : (2, 1)
(9, 5) : (2, 1)
(9, 6) : (2, 1)
(9, 7) : (2, 1)
(9, 8) : (2, 2)
(9, 9) : (2, 2)
(9, 10) : (2, 2)
(9, 11) : (2, 2)



### When $k$ doesn't divide $n$
Now I want to test what happens when $k$ does not divide $n$ and hence we sample an origin point $(q_1,q_2).$

In [6]:
from numpy.random import default_rng
n = 7
k = 2
print(n%k == 0)

False


In [7]:
# sample origin
rng = default_rng()
q1 = rng.integers(low=0, high=k)
q2 = rng.integers(low=0, high=k)
print(q1,q2)

1 0


In [8]:
num = np.ceil(n/k)
print('Cluster',':','I  ','  J')
for i in range(num.astype(int)):
    for j in range(num.astype(int)):
        I,J = ds.sqlat_toUnit(i,j,k,n,q1,q2)
        print((i,j),':', I,J)
    print()

Cluster : I     J
(0, 0) : [0 1] [0]
(0, 1) : [0 1] [1 2]
(0, 2) : [0 1] [3 4]
(0, 3) : [0 1] [5 6]

(1, 0) : [2 3] [0]
(1, 1) : [2 3] [1 2]
(1, 2) : [2 3] [3 4]
(1, 3) : [2 3] [5 6]

(2, 0) : [4 5] [0]
(2, 1) : [4 5] [1 2]
(2, 2) : [4 5] [3 4]
(2, 3) : [4 5] [5 6]

(3, 0) : [6] [0]
(3, 1) : [6] [1 2]
(3, 2) : [6] [3 4]
(3, 3) : [6] [5 6]



In [9]:
q1 = rng.integers(low=0, high=k)
q2 = rng.integers(low=0, high=k)
print(q1,q2)

0 0


In [10]:
num = np.ceil(n/k)
print('Cluster',':','I  ','  J')
for i in range(num.astype(int)):
    for j in range(num.astype(int)):
        I,J = ds.sqlat_toUnit(i,j,k,n,q1,q2)
        print((i,j),':', I,J)
    print()

Cluster : I     J
(0, 0) : [0] [0]
(0, 1) : [0] [1 2]
(0, 2) : [0] [3 4]
(0, 3) : [0] [5 6]

(1, 0) : [1 2] [0]
(1, 1) : [1 2] [1 2]
(1, 2) : [1 2] [3 4]
(1, 3) : [1 2] [5 6]

(2, 0) : [3 4] [0]
(2, 1) : [3 4] [1 2]
(2, 2) : [3 4] [3 4]
(2, 3) : [3 4] [5 6]

(3, 0) : [5 6] [0]
(3, 1) : [5 6] [1 2]
(3, 2) : [5 6] [3 4]
(3, 3) : [5 6] [5 6]

